# Entry & Exit Signal

- Create separate notebook as 'poc.ipynb' notebook is too bloated.
- Create abstract class


In [1]:
import pandas as pd
from abc import ABC, abstractmethod
from datetime import datetime, date
import re
from collections import deque, Counter
from decimal import Decimal
import numpy as np
import yfinance as yf
from pydantic import BaseModel, Field, computed_field, field_validator
import math
from pathlib import Path
from zoneinfo import ZoneInfo

%load_ext autoreload
%autoreload 2
%matplotlib inline

from src.utils import utils
from config.variables import EntryType, COINT_CORR_FN, STRUCT_MAPPING
from src.strategy.base import StockTrade
from src.strategy.base import MultiHalfEntry, MultiEntry, SingleEntry, FIFOExit, LIFOExit, HalfFIFOExit

In [5]:
trade = StockTrade(
    ticker="AAPL",
    entry_datetime="2025-04-01",
    entry_action="buy",
    entry_lots=10,
    entry_price=200
)
trade1 = StockTrade(
    ticker="AAPL",
    entry_datetime="2025-04-02",
    entry_action="buy",
    entry_lots=10,
    entry_price=195
)

open_trades = deque([trade, trade1])
open_trades

deque([StockTrade(ticker='AAPL', entry_datetime=datetime.datetime(2025, 4, 1, 0, 0), entry_action='buy', entry_lots=Decimal('10'), entry_price=Decimal('200'), exit_datetime=None, exit_action=None, exit_lots=Decimal('0'), exit_price=None, days_held=None, profit_loss=None, percent_ret=None, daily_ret=None, win=None),
       StockTrade(ticker='AAPL', entry_datetime=datetime.datetime(2025, 4, 2, 0, 0), entry_action='buy', entry_lots=Decimal('10'), entry_price=Decimal('195'), exit_datetime=None, exit_action=None, exit_lots=Decimal('0'), exit_price=None, days_held=None, profit_loss=None, percent_ret=None, daily_ret=None, win=None)])

In [6]:
class_name = STRUCT_MAPPING.get("multiple")
entry_method = utils.get_class_instance(class_name, "./src/strategy/base/entry_struct.py", num_lots=20)
open_trades = entry_method.open_new_pos(open_trades, "AAPL", "2025-04-04", "buy", 201)
open_trades

deque([StockTrade(ticker='AAPL', entry_datetime=datetime.datetime(2025, 4, 1, 0, 0), entry_action='buy', entry_lots=Decimal('10'), entry_price=Decimal('200'), exit_datetime=None, exit_action=None, exit_lots=Decimal('0'), exit_price=None, days_held=None, profit_loss=None, percent_ret=None, daily_ret=None, win=None),
       StockTrade(ticker='AAPL', entry_datetime=datetime.datetime(2025, 4, 2, 0, 0), entry_action='buy', entry_lots=Decimal('10'), entry_price=Decimal('195'), exit_datetime=None, exit_action=None, exit_lots=Decimal('0'), exit_price=None, days_held=None, profit_loss=None, percent_ret=None, daily_ret=None, win=None),
       StockTrade(ticker='AAPL', entry_datetime=datetime.datetime(2025, 4, 4, 0, 0), entry_action='buy', entry_lots=Decimal('20'), entry_price=Decimal('201'), exit_datetime=None, exit_action=None, exit_lots=Decimal('0'), exit_price=None, days_held=None, profit_loss=None, percent_ret=None, daily_ret=None, win=None)])

In [17]:
class_name = STRUCT_MAPPING.get("half_fifo")
exit_method = utils.get_class_instance(class_name, "./src/strategy/base/exit_struct.py")                 
open_trades, completed_trades = exit_method.close_pos(open_trades, "2025-04-19", "sell", 229)
open_trades

deque([])

In [18]:
completed_trades

[{'ticker': 'AAPL',
  'entry_datetime': datetime.datetime(2025, 4, 4, 0, 0),
  'entry_action': 'buy',
  'entry_lots': Decimal('1'),
  'entry_price': Decimal('201'),
  'exit_datetime': datetime.datetime(2025, 4, 19, 0, 0),
  'exit_action': 'sell',
  'exit_lots': Decimal('1'),
  'exit_price': Decimal('229'),
  'days_held': 15,
  'profit_loss': Decimal('28'),
  'percent_ret': Decimal('0.139303'),
  'daily_ret': Decimal('0.008732'),
  'win': 1}]

In [3]:
date = "2025-04-01"
model = "ziweichen_coint_5"
ticker_pair = "NFLX_EA"

results_dir = f"./data/results/{date}"
model_dir = f"{results_dir}/{model}"
pa_dir = f"{model_dir}/price_actions"

In [ ]:
df = utils.load_csv(f"{pa_dir}/{ticker_pair}.csv")
# df = df.set_index("date")
df

In [4]:
EA = pd.read_parquet("./data/stock/EA.parquet")
EA

,Ticker,Open,High,Low,Close,Volume
Date,,,,,,
2020-01-02,EA,105.30,105.70,104.15,104.71,1901000
2020-01-03,EA,103.00,105.11,102.55,104.58,1840300
2020-01-06,EA,104.33,106.64,104.02,106.12,2934200
2020-01-07,EA,106.33,106.58,105.09,105.74,1692400
2020-01-08,EA,105.53,107.31,105.14,106.81,2651600
...,...,...,...,...,...,...
2025-03-21,EA,142.27,145.22,142.00,144.30,9254800
2025-03-24,EA,145.78,145.87,143.57,144.55,2848100
2025-03-25,EA,145.28,146.45,143.77,144.50,2382000


In [ ]:
EA.index = pd.to_datetime(EA.index)
df.index = pd.to_datetime(df.index)

df[["coint_corr_ticker", "open", "high", "low", "close"]] = EA.loc[
    EA.index.isin(df.index), ["Ticker", "Open", "High", "Low", "Close"]
]

df

,ticker,day_name,is_holiday,av_rating,median_rating_excl,NFLX_close,EA_close,action,coint_corr_ticker,open,high,low,close
date,,,,,,,,,,,,,
2025-03-01,NFLX,Saturday,False,3.622950819672131,5.0,NaN,NaN,buy,NaN,NaN,NaN,NaN,NaN
2025-03-02,NFLX,Sunday,False,5.0,5.0,NaN,NaN,buy,NaN,NaN,NaN,NaN,NaN
2025-03-03,NFLX,Monday,False,3.0,3.0,973.7,129.23,wait,EA,130.38,131.60,128.87,129.23
2025-03-04,NFLX,Tuesday,False,3.0,3.0,972.58,131.82,wait,EA,129.52,133.35,129.24,131.82
2025-03-05,NFLX,Wednesday,False,3.25,4.0,990.92,134.05,buy,EA,131.60,134.51,131.31,134.05
2025-03-06,NFLX,Thursday,False,4.5,5.0,906.36,136.79,buy,EA,133.65,137.53,133.37,136.79
2025-03-07,NFLX,Friday,False,3.0,3.0,891.11,140.04,wait,EA,136.19,140.42,136.00,140.04
2025-03-08,NFLX,Saturday,False,2.75,2.5,NaN,NaN,wait,NaN,NaN,NaN,NaN,NaN
2025-03-09,NFLX,Sunday,False,3.0,3.0,NaN,NaN,wait,NaN,NaN,NaN,NaN,NaN
